In [1]:
#from tensorflow.python import keras
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
config = tf.compat.v1.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

#from keras import backend
#backend.set_session(session)
#print(backend._get_available_gpus())

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.__version__

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7989803587745975418
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5651511826185801252
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10967262691985611617
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2435186688
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6877443409011915869
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5

'2.3.0'

In [2]:
from keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.models import Sequential, Input, Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.python.keras.layers import Dot, Average, Dense, Dropout, RepeatVector, Conv2D, Flatten, MaxPooling2D, TimeDistributed, BatchNormalization, LSTM, Concatenate, Embedding, Bidirectional
from keras.preprocessing.sequence import pad_sequences

from random import randint, choice

from sklearn.utils import class_weight
import numpy as np

import pickle


#import nltk
#nltk.download('averaged_perceptron_tagger')

import sys
sys.path.insert(1, '../utils')
from NLP_little_helpers import *
little_helpers()

[nltk_data] Downloading package brown to /home/felipe/nltk_data...
[nltk_data]   Package brown is already up-to-date!


potions: re, numpy as np, pandas as pd, pickle, json, nltk, keras, collections
spells: clean_data, clean_entire, predict_tags, tagged_3D, tagged_n_grams, unknown_words_X, check_and_predict


In [3]:
def n_grams(X, X_tag, n=4):
    """
    This function takes a sequence of tupled sentences and tags:
    [('Here is an example', 'tag1 tag2 tag3 tag4')]
    Returns a tuple with other two tuples inside and the target (y).
    The first tuple is for the encoded texts and the second for the encoded tags.
    Each tuple contains two arrays:
    1. The first array contains padded n-grams combinations(X).
    2. The second array is composed of the next n words.
    
    All returned sequences are already pre-padded with 0.
    The maximum length of the sequences is n, with n being the desired n-grams.
    """  
    
    text_sequences = list()
    tags_sequences = list()
    
    for i in range(len(X)):
        text_sent_sequences = list()
        tags_sent_sequences = list()
        first_sent_words = list()
        first_sent_tags = list()
        
        for j in range(len(X[i])):
            
            first_words = X[i][:j+2]
            first_tags = X_tag[i][:j+2]
            text_sequence = X[i][j:j+n+1]
            tags_sequence = X_tag[i][j:j+n+1]
            # avoinding repeated n-grams in the same sentence
            if text_sequence not in text_sent_sequences and len(text_sequence)>1 and len(tags_sequence)>1 and len(text_sequence)<=n:
                text_sent_sequences += [text_sequence]
                tags_sent_sequences += [tags_sequence]
                first_sent_words += [first_words]
                first_sent_tags += [first_tags]

        if len(text_sent_sequences)>1:
            text_sequences += first_sent_words + text_sent_sequences
            tags_sequences += first_sent_tags + tags_sent_sequences

    print('Total Sequences: %d' % len(text_sequences))
    X = pad_sequences([seq[:-1] for seq in text_sequences], maxlen=n)
    X_tag = pad_sequences([seq[:-1] for seq in tags_sequences], maxlen=n)
    y = np.array([[seq[-1]] for seq in text_sequences])
    assert len(X) == len(y) == len(X_tag)
    
    return X, X_tag, y



In [4]:
with open('../Data/quotes_tagged.json', 'r') as outfile:
    viterbis_dict = json.load(outfile)
      
text = [' '.join([['<bos>'] + words + ['<eos>']][0]) for words, tags in viterbis_dict if tags != 'UNK']
tags = [' '.join([['<bos>'] + tags + ['<eos>']][0]) for words, tags in viterbis_dict if tags != 'UNK']

# integer encode sequences of words
tokenizer = Tokenizer(filters='', lower=False)
tokenizer.fit_on_texts(text+tags)
enc_texts = tokenizer.texts_to_sequences(text)
enc_tags = tokenizer.texts_to_sequences(tags)

seq2word_map = dict(map(reversed, tokenizer.word_index.items()))

with open('../Trained_Weights/tokenizer_dict.json', 'w') as file:
    json.dump(seq2word_map, file)
    
#with open('../Trained_Weights/tokenizer_text.pkl', 'wb') as file:
#    pickle.dump(t_text, file)
    
#with open('../Trained_Weights/tokenizer_tags.pkl', 'wb') as file:
#    pickle.dump(t_tags, file)

#max_length = max([len(sent) for sent in enc_texts])
tags_size = len(set([t for tag in tags for t in tag]))
vocab_size = len(set([word for words in enc_texts for word in words]))
embed_size = tags_size+vocab_size

X_text, X_tags, y = n_grams(enc_texts, enc_tags, n=5)

max_length = X_text.shape[1]

print(text[0], '\n', tags[0])
print('Vocabulary Size: %d' % vocab_size)
print('Unique Tags: %d' % tags_size)
print('Max length:', max_length)

y_weight = [j for i in y for j in i]
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_weight), y_weight)

Total Sequences: 4158864
<bos> you know you are in love when you cannot fall asleep because reality is finally better than your dreams <eos> 
 <bos> PPSS VB PPSS BER IN NN WRB PPSS MD* VB RB CS NN BEZ RB RBR CS PP$ NNS <eos>
Vocabulary Size: 183296
Unique Tags: 42
Max length: 5


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
# Glove embedding

embeddings_dict = {}
with open("../Data/glove.6B.300d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [6]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [7]:
len(embedding_matrix)

183534

In [8]:
embedding_matrix[76]
X_text[10]
tokenizer.word_index['the']

9

In [14]:
X_text1 = [list(x) for x in X_text]
X_tags1 = [list(x) for x in X_tags]
X = list(zip(X_text1, X_tags1))
X = [list(x) for x in X]

In [10]:
from gensim.models import Word2Vec

text = [[['<bos>'] + words + ['<eos>']][0] for words, tags in viterbis_dict if tags != 'UNK']

tags = [[['<bos>'] + tags + ['<eos>']][0] for words, tags in viterbis_dict if tags != 'UNK']

# embedding words with word2vec (skip-gram)
embedded = Word2Vec(tags+text, sg=1, min_count=0, size=300)

In [11]:
embedding_matrix_w2v = np.zeros((len(tokenizer.word_index.items())+1, 300))
for word, i in tokenizer.word_index.items():
    embedding_matrix_w2v[i] = embedded.wv[word]

In [15]:
X_new = []
for text, tags in X:
    for i in range(len(text)):
        text[i] = embedding_matrix[text[i]]
        tags[i] = embedding_matrix_w2v[tags[i]]
        X_new += [np.multiply(text[i], tags[i]/2)]

In [16]:
X_new.shape

(300,)

In [13]:
import tensorflow as tf

inputs = tf.python.keras.Input((2,5,300))
model_text = tf.python.keras.layers.Flatten()(inputs)
model_text = tf.python.keras.layers.RepeatVector(1)(model_text)
model_text = tf.python.keras.layers.Bidirectional(tf.python.keras.layers.CuDNNLSTM(8))(model_text)
result = tf.python.keras.layers.Dense(vocab_size+1, activation='softmax')(model_text)
model_text = tf.python.keras.Model(inputs, result)

model_text.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_text.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2, 5, 300)]       0         
_________________________________________________________________
flatten (Flatten)            (None, 3000)              0         
_________________________________________________________________
repeat_vector (RepeatVector) (None, 1, 3000)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16)                192640    
_________________________________________________________________
dense (Dense)                (None, 183297)            3116049   
Total params: 3,308,689
Trainable params: 3,308,689
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_text.fit(X, y,
          batch_size=4, epochs=1, verbose=1,
          class_weight = class_weights,
          shuffle=True, validation_split=0.2,
          workers=8, use_multiprocessing=True)

In [ ]:
model_text.save('../Trained_Weights/tagged_quotes.h5')

In [20]:
# using n-grams of sentences, tags, reversed sentences and reversed tags
# concatenating each model to get a more reliable prediction 
with tf.device('/cpu:0'):  
    text_input = Input((max_length,))
    model_text = Embedding(len(embedding_matrix), 300, weights=[embedding_matrix], trainable=False)(text_input)

#model_text = LSTM(8, return_sequences=True, dropout=0.2, activation='sigmoid')(model_text)
#model_text = TimeDistributed(Dense(1, activation='selu'))(model_text)
#model_text = LSTM(4, dropout=0.2)(model_text)

tags_input = Input((max_length,))
model_tags = Embedding(tags_size, 300)(tags_input)
#model_tags = LSTM(8, return_sequences=True, dropout=0.2, activation='sigmoid')(model_tags)
#model_tags = TimeDistributed(Dense(1, activation='selu'))(model_tags)
#model_tags = LSTM(4, dropout=0.2)(model_tags)

concatenate = Average()([model_text, model_tags])
"""
rev_text_input = Input((max_length,))
rev_model_text = Embedding(embed_size, 64)(rev_text_input)
rev_model_text = Bidirectional(LSTM(8, dropout=0.2, recurrent_dropout=0.1))(rev_model_text)

rev_tags_input = Input((max_length,))
rev_model_tags = Embedding(embed_size, 64)(rev_tags_input)
rev_model_tags = Bidirectional(LSTM(8, dropout=0.2, recurrent_dropout=0.1))(rev_model_tags)

rev_concatenate_text_tags = Concatenate()([rev_model_text, rev_model_tags])

concatenate = Concatenate()([concatenate_text_tags, rev_concatenate_text_tags])

"""

#model = RepeatVector(1)(concatenate)
model = Bidirectional(LSTM(2, dropout=0.2, return_sequences=True, activation='selu'))(concatenate)
model = TimeDistributed(Dense(1, activation='selu'))(model)
model = Bidirectional(LSTM(2, dropout=0.2, activation='selu'))(model)

result = Dense(vocab_size+2, activation='softmax')(model)

model = Model(inputs=[text_input, tags_input], outputs=result)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 5, 300)       54989400    input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 5, 300)       72000       input_4[0][0]                    
__________________________________________________________________________________________________
average (A

In [22]:
callbacks=[EarlyStopping(patience=1, monitor='val_loss'),
           ModelCheckpoint(filepath='model_last.{epoch:02d}-{val_loss:.2f}.h5')]

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit([X, X_tags], y,
          callbacks=callbacks,
          batch_size=64, epochs=2, verbose=1,
          class_weight = class_weights,
          shuffle=True, validation_split=0.2,
          workers=8, use_multiprocessing=True)

Train on 3327091 samples, validate on 831773 samples
Epoch 1/2
3327091/3327091 [==============================] - 4603s 1ms/sample - loss: 2.8410 - acc: 0.6292 - val_loss: 2.8444 - val_acc: 0.6345
Epoch 2/2
3327091/3327091 [==============================] - 3772s 1ms/sample - loss: 2.7305 - acc: 0.6340 - val_loss: 2.8338 - val_acc: 0.6351


In [24]:
model.save('../Trained_Weights/tagged_quotes.h5')

### ConvNets

In [ ]:
with open('../Data/100_books_tagged.json', 'r') as outfile:
    viterbis_dict = json.load(outfile)
      
text = [[['<bos>'] + words + ['<eos>']][0] for words, tags in viterbis_dict if tags != 'UNK']
tags = [[['<bos>'] + tags + ['<eos>']][0] for words, tags in viterbis_dict if tags != 'UNK']


# integer encode sequences of words
# integer encode sequences of words
t_text = Tokenizer(filters='', oov_token=1)
t_text.fit_on_texts(text)
enc_texts = t_text.texts_to_sequences(text)

t_tags = Tokenizer(filters='', oov_token=1)
t_tags.fit_on_texts(tags)
enc_tags = t_tags.texts_to_sequences(tags)

seq2word_map = dict(map(reversed, t.word_index.items()))

with open('../Trained_Weights/tokenizer_dict.json', 'w') as file:
    json.dump(seq2word_map, file)
    
with open('../Trained_Weights/tokenizer.pkl', 'w') as file:
    pickle.dump(t, file)

#max_length = max([len(sent) for sent in enc_texts])
tags_size = len(set([t for tag in tags for t in tag]))
vocab_size = len(set([word for words in enc_texts for word in words]))
embed_size = tags_size+vocab_size

X, X_tags, y = n_grams(enc_texts, enc_tags, n=20)

max_length = X.shape[1]

print('Vocabulary Size: %d' % vocab_size)
print('Unique Tags: %d' % tags_size)
print('Max length:', max_length)

In [44]:
X_2D = np.array(list(zip(X, X_tags)))

In [45]:
model_text = Sequential()
model_text.add(Embedding(vocab_size+1, 64, input_shape=(2,max_length)))

model_text.add(Conv2D(512, kernel_size=4, padding='same', activation='relu'))
model_text.add(Dropout(0.2))
model_text.add(MaxPooling2D(2, 4, padding='same'))
model_text.add(BatchNormalization())
model_text.add(Flatten())
model_text.add(Dense(vocab_size/12, activation='selu'))
#model_text.add(Dense(1, activation='selu'))
model_text.add(RepeatVector(1))
model_text.add(Bidirectional(LSTM(128, dropout = 0.2, return_sequences=True)))

"""
model_text.add(Conv2D(8, kernel_size=3, padding='same', activation='relu'))
model_text.add(Dropout(0.5))
model_text.add(Conv2D(4, kernel_size=4, padding='same', activation='relu'))
model_text.add(BatchNormalization())
model_text.add(Flatten())
model_text.add(Dropout(0.5))
"""

model_text.add(TimeDistributed(Dense(vocab_size+1, activation='softmax')))

model_text.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_31 (Embedding)     (None, 2, 5, 64)          5516800   
_________________________________________________________________
conv2d (Conv2D)              (None, 2, 5, 512)         524800    
_________________________________________________________________
dropout (Dropout)            (None, 2, 5, 512)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 2, 512)         0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 1, 2, 512)         2048      
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense_34 (Dense)             (None, 7183)              7362575   
__________

In [ ]:
callbacks=[EarlyStopping(patience=1, monitor='val_loss'),
           ModelCheckpoint(filepath='model_last.{epoch:02d}-{val_loss:.2f}.h5')]

model_text.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_text.fit(X_2D, y,
          callbacks=callbacks,
          batch_size=32, epochs=3, verbose=1,
          shuffle=True, validation_split=0.2,
          workers=8, use_multiprocessing=True)

In [ ]:
model_text.save('conv_100_books.h5')

In [10]:
embedding_matrix.shape

(59336, 50)

In [13]:
with tf.device('/cpu:0'):      
    model_text = Sequential()
    model_text.add(Embedding(len(embedding_matrix), 300, input_length=max_length,
                             weights=[embedding_matrix], trainable=False))

model_text.add(Bidirectional(LSTM(4, dropout = 0.2)))
#model_text.add(TimeDistributed(Dense(1, activation='selu')))
#model_text.add(LSTM(8, dropout = 0.2))
model_text.add(Dense(vocab_size+1, activation='softmax'))

model_text.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 5, 300)            25688400  
_________________________________________________________________
bidirectional (Bidirectional (None, 8)                 9760      
_________________________________________________________________
dense_4 (Dense)              (None, 85627)             770643    
Total params: 26,468,803
Trainable params: 780,403
Non-trainable params: 25,688,400
_________________________________________________________________


In [14]:
callbacks=[EarlyStopping(patience=1, monitor='val_loss'),
           ModelCheckpoint(filepath='model_last.{epoch:02d}-{val_loss:.2f}.h5')]

model_text.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_text.fit(X, y,
          #callbacks=callbacks,
          batch_size=32, epochs=3, verbose=1,
          shuffle=True, validation_split=0.2,
          workers=8, use_multiprocessing=True)

Train on 639971 samples, validate on 159993 samples
Epoch 1/3
639971/639971 [==============================] - 821s 1ms/sample - loss: 3.1178 - acc: 0.6248 - val_loss: 2.8764 - val_acc: 0.6243
Epoch 2/3
639971/639971 [==============================] - 460s 718us/sample - loss: 2.8093 - acc: 0.6264 - val_loss: 2.8129 - val_acc: 0.6264
Epoch 3/3
639971/639971 [==============================] - 451s 705us/sample - loss: 2.7592 - acc: 0.6280 - val_loss: 2.7900 - val_acc: 0.6285


In [ ]:
text[:2]

In [3]:
def text_grams(X, n=4):
    """
    This function takes a sequence of tupled sentences and tags:
    [('Here is an example', 'tag1 tag2 tag3 tag4')]
    Returns a tuple with other two tuples inside and the target (y).
    The first tuple is for the encoded texts and the second for the encoded tags.
    Each tuple contains two arrays:
    1. The first array contains padded n-grams combinations(X).
    2. The second array is composed of the next n words.
    
    All returned sequences are already pre-padded with 0.
    The maximum length of the sequences is n-1, with n being the desired n-grams.
    """  
    
    text_sequences = list()
    
    for i in range(len(X)):
        text_sent_sequences = list()
        first_sent_words = list()
        
        for j in range(len(X[i])):
            
            first_words = X[i][:j+2]
            text_sequence = X[i][j:j+n+1]
            # avoinding repeated n-grams in the same sentence
            if text_sequence not in text_sent_sequences and len(text_sequence)>1 and len(text_sequence)<=n:
                text_sent_sequences += [text_sequence]
                first_sent_words += [first_words]

        if len(text_sent_sequences)>1:
            text_sequences += first_sent_words + text_sent_sequences 

    print('Total Sequences: %d' % len(text_sequences))
    X = pad_sequences([seq[:-1] for seq in text_sequences], maxlen=n)
    y = np.array([[seq[-1]] for seq in text_sequences])
    assert len(X) == len(y)
    
    return X, y


In [4]:
with open('../Data/quotes_clean.pkl', 'rb') as file:
    corpus = pickle.load(file)
      
text = [[['<bos>'] + words.split() + ['<eos>']][0] for words in corpus[:100000]]

# integer encode sequences of words
t_text = Tokenizer(filters='', oov_token=1)
t_text.fit_on_texts(text)
enc_texts = t_text.texts_to_sequences(text)

seq2word_map = dict(map(reversed, t_text.word_index.items()))

with open('../Trained_Weights/tokenizer_dict.json', 'w') as file:
    json.dump(seq2word_map, file)
    
with open('../Trained_Weights/tokenizer_text.pkl', 'wb') as file:
    pickle.dump(t_text, file)
    
#max_length = max([len(sent) for sent in enc_texts])
vocab_size = len(set([word for words in enc_texts for word in words]))

X, y = text_grams(enc_texts, n=5)

max_length = X.shape[1]

print('Vocabulary Size: %d' % vocab_size)
print('Max length:', max_length)

Total Sequences: 799964
Vocabulary Size: 85626
Max length: 5


In [5]:
# Glove embedding

embeddings_dict = {}
with open("../Data/glove.6B.300d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

        
embedding_matrix = np.zeros((len(t_text.word_index) + 1, 300))

for word, i in t_text.word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
# using n-grams of sentences, tags, reversed sentences and reversed tags
# concatenating each model to get a more reliable prediction 
with tf.device('/cpu:0'):  
    text_input = Input((max_length,))
    model_text = Embedding(len(embedding_matrix), 50, weights=[embedding_matrix], trainable=False)(text_input)
    model_text = TimeDistributed(Dense(len(embedding_matrix), activation = 'linear'))(model_text)

model_text = Bidirectional(LSTM(32, return_sequences=True, dropout=0.2))(model_text)
model_text = TimeDistributed(Dense(1000, activation='selu'))(model_text)
model_text = Bidirectional(LSTM(8, return_sequences=True, dropout=0.2))(model_text)

#tags_input = Input((max_length,))
#model_tags = Embedding(tags_size, 32)(tags_input)
#model_tags = Bidirectional(LSTM(8, dropout=0.2))(model_tags)
#model_tags = TimeDistributed(Dense(1, activation='selu'))(model_tags)
#model_tags = Bidirectional(LSTM(4, dropout=0.2))(model_tags)

#concatenate = Concatenate()([model_text, model_tags])
"""
rev_text_input = Input((max_length,))
rev_model_text = Embedding(embed_size, 64)(rev_text_input)
rev_model_text = Bidirectional(LSTM(8, dropout=0.2, recurrent_dropout=0.1))(rev_model_text)

rev_tags_input = Input((max_length,))
rev_model_tags = Embedding(embed_size, 64)(rev_tags_input)
rev_model_tags = Bidirectional(LSTM(8, dropout=0.2, recurrent_dropout=0.1))(rev_model_tags)

rev_concatenate_text_tags = Concatenate()([rev_model_text, rev_model_tags])

concatenate = Concatenate()([concatenate_text_tags, rev_concatenate_text_tags])

"""

#model = TimeDistributed(Dense(vocab_size+2, activation='selu'))(concatenate)
#model = Bidirectional(LSTM(4, dropout=0.2))(model)
#rpt = RepeatVector(1)(concatenate)

result = TimeDistributed(Dense(vocab_size+2, activation='softmax'))(model_text)

model = Model(inputs=text_input, outputs=result)

model.summary()

In [ ]:
callbacks=[EarlyStopping(patience=1, monitor='val_loss'),
           ModelCheckpoint(filepath='model_last.{epoch:02d}-{val_loss:.2f}.h5')]

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y,
          callbacks=callbacks,
          batch_size=32, epochs=5, verbose=1,
          #shuffle=True, validation_split=0.2,
          workers=8, use_multiprocessing=True)

In [ ]:
model.save('quotes.h5')

In [ ]:
X[:1], y[:1]

In [ ]:
def create_class_weight(labels_dict,mu=0.15):
    total = np.sum(labels_dict.values())
    keys = labels_dict.keys()
    class_weight = dict()

    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0

    return class_weight